In [127]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from joblib import dump, load

In [128]:
data_train = pd.read_csv('./data/train.csv', index_col=0)
data_test = pd.read_csv('./data/test.csv', index_col=0)
data_responses = pd.read_csv('./data/gender_submission.csv', index_col=0)
data_test = pd.concat([data_test, data_responses], axis = 1)
#data = pd.concat([data_train, data_test])
data = data_train

In [129]:
data.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [130]:
data['Cabin'] = data['Cabin'].fillna('Unknown')
data['Age'] = data['Age'].fillna(data['Age'].mean())
data['Embarked'] = data['Embarked'].fillna('U')
data['Fare'] = data['Fare'].fillna(data['Fare'].mean())

In [131]:
data.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [132]:
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Survived']

data_cleaned = data[columns]

In [133]:
data_cleaned.Pclass=np.where(data_cleaned.Pclass== 1,'First',
                   np.where(data_cleaned.Pclass==2,
                            'Second', 'Third'))
data_cleaned['Cabin_class'] = data_cleaned.Cabin.str[0]
data_cleaned['isfemale']=np.where(data_cleaned.Sex=='female', 1, 0)
data_cleaned.drop(['Sex', 'Cabin'], axis = 1, inplace=True)

/Users/german.dominguez/.pyenv/versions/Titanic_GDG/lib/python3.9/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-133-b7e494252a36>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Cabin_class'] = data_cleaned.Cabin.str[0]


In [134]:
data_cleaned_dummies = pd.get_dummies(data_cleaned, drop_first=True,
                                      columns=['Pclass', 'Cabin_class', 'Embarked'])

In [135]:
X = data_cleaned_dummies.drop('Survived', axis= 1)
y = data_cleaned_dummies.Survived

In [136]:
#logreg = LogisticRegression(solver='liblinear', max_iter=200, random_state=42)
logreg = LogisticRegression(solver='sag', max_iter=100, random_state=42)
logreg.fit(X, y).coef_

/Users/german.dominguez/.pyenv/versions/Titanic_GDG/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


array([[-0.02616928, -0.08470487, -0.01032506,  0.01501974,  0.13723528,
         0.02839064, -0.0804756 ,  0.01062565,  0.00072936,  0.01528716,
         0.0160185 ,  0.00386405,  0.00032862, -0.00048928, -0.06456776,
         0.00566744, -0.04372611,  0.00158441]])

In [137]:
#rfe = RFE(logreg, 19)
#rfe = rfe.fit(X, y)
#rfe.support_
#rfe.ranking_
#columnas_RFE = np.array(X.columns)[rfe.support_]
#X = X[columnas_RFE]

In [138]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [139]:
logreg.fit(X_train, y_train)

/Users/german.dominguez/.pyenv/versions/Titanic_GDG/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(random_state=42, solver='sag')

In [140]:
y_pred = logreg.predict(X_test)
logreg.score(X_test, y_test)

0.6815642458100558

In [141]:
matriz_confusion = confusion_matrix(y_test, y_pred)
matriz_confusion

array([[101,   4],
       [ 53,  21]])

In [142]:
data_cleaned.describe()

,Age,SibSp,Parch,Fare,Survived,isfemale
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,0.523008,0.381594,32.204208,0.383838,0.352413
std,13.002015,1.102743,0.806057,49.693429,0.486592,0.477990
min,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000
50%,29.699118,0.000000,0.000000,14.454200,0.000000,0.000000
75%,35.000000,1.000000,0.000000,31.000000,1.000000,1.000000
max,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000


In [143]:
set(data_cleaned.Pclass)

{'First', 'Second', 'Third'}

In [144]:
set(data_cleaned.Cabin_class)

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'U'}

In [145]:
set(data_cleaned.Embarked)


{'C', 'Q', 'S', 'U'}

In [146]:
dump(logreg, './titanic_logreg.joblib')

['./titanic_logreg.joblib']

In [147]:
pasajero = [[48, 0, 0, 300.0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]]
predict = logreg.predict(pasajero)
predict[0]

1